In [ ]:
%matplotlib inline
import sklearn
import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display

import IPython.display as ipd

import warnings
warnings.filterwarnings('ignore')

import import_ipynb
import preprocessor
import display
import classifier
import csv_file_io
import datetime

PROJECT_HOME = './'
# DRONE = 'Inspire2' # 'Bebop2', 'Spark', 'MavicPro', 'Phantom4', 'Matrice100', 'MavicAir', 'AutelEVO', 'Inspire2', 'JME', 'ParrotAnafi']
# DATASET_DIR = PROJECT_HOME + 'dataset/validation/Inspire2/Inspire2_validation_File3.wav'
RESULT_FILE_DIR = PROJECT_HOME + 'results/'
DATASET_FILE_DIR = PROJECT_HOME + 'results/dataset/'
# TRAINING_DATASET_DIR = PROJECT_HOME + 'dataset/1. training/'
# VALIDATION_DATASET_DIR = PROJECT_HOME + 'dataset/2. validation/'
# TEST_DATASET_DIR = PROJECT_HOME + 'dataset/3. test/'
# TRAINING_VALIDATION_DATASET_DIR = PROJECT_HOME + 'dataset/4. training_validation/'
# TRAINING_VALIDATION_TEST_DATASET_DIR = PROJECT_HOME + 'dataset/5. training_validation_test/'
SEGMENTATION_DATASET_DIR = PROJECT_HOME + 'dataset/6. segmentation/'
# SEGMENTATION_TEST_DATASET_DIR = PROJECT_HOME + 'dataset/7. segmentation(test)/'

now = datetime.datetime.now()
now_date_time = now.strftime('%Y%m%d_%H%M%S')[2:]

FILE_NAME = "[Training, Validation, Test] Classification result ()"

SAMPLE_RATE = 44100
FIG_SIZE = (15,5)

FRAME_LENGTH = 0.250 # 250 ms
FRAME_STRIDE = 0.250 # same with frame_length for 0% overlap

DIGITS = 3

b_visualizing = False

def save_each_classification_performance(kernel: str, target: list, acc: float, tpr: float, fnr: float, tnr: float, fpr: float):
    file_name = FILE_NAME.split(')')[0] + kernel + ') '

    f = csv_file_io.open_csv(RESULT_FILE_DIR + file_name + now_date_time + '.csv', 'w')
    csv_file_io.write_csv_data(f, ['Kernel', kernel])
    csv_file_io.write_csv_data(f, ['ACC', acc])
    csv_file_io.write_csv_data(f, ['', 'TPR', 'FNR', 'TNR', 'FPR'])
    
    for i in range (0, len(target)):
        csv_file_io.write_csv_data(f, [target[i], tpr[i], fnr[i], tnr[i], fpr[i]])    

    csv_file_io.write_csv_data(f, [])
    csv_file_io.write_csv_data(f, ['Average', round(np.mean(tpr), DIGITS), round(np.mean(fnr), DIGITS), round(np.mean(tnr), DIGITS), round(np.mean(fpr), DIGITS)])

    f.close()

    # f = csv_file_io.open_csv(RESULT_FILE_DIR + file_name + now_date_time + '.csv', 'a')
    # csv_file_io.write_csv_data(f, [kernel, acc, tpr, fnr, tnr, fpr])
    # f.close()


def save_total_classification_performance(res_dict: dict):
    file_name = FILE_NAME.split(')')[0] + 'total) '
    f = csv_file_io.open_csv(RESULT_FILE_DIR + file_name + now_date_time + '.csv', 'w')
    csv_file_io.write_csv_data(f, ['Kernel', 'ACC', 'TPR', 'FNR', 'TNR', 'FPR'])
    # f.close()

    for i in range(0, len(res_dict['kernel'])):
        # f = csv_file_io.open_csv(RESULT_FILE_DIR + FILE_NAME + now_date_time + '.csv', 'a')
        csv_file_io.write_csv_data(f, [res_dict['kernel'][i], res_dict['acc'][i],  res_dict['tpr'][i], res_dict['fnr'][i], res_dict['tnr'][i], res_dict['fpr'][i]])    

    f.close()

In [ ]:
dict_integrated_train_validation_signal = preprocessor.get_integrated_signal(TRAINING_VALIDATION_DATASET_DIR, type='TRAINING+VALIDATION')
dict_integrated_test_signal = preprocessor.get_integrated_signal(TEST_DATASET_DIR, type='TEST')

# Visualizing Audio (waveform and power spectrum)
if b_visualizing == True:
    for key, val in dict_integrated_train_validation_signal.items():
        display.display_waveshow(val, key + "'s Training+Validation Signal's Waveform")
        display.display_power_spectrum(val, key + "'s Training+Validation Signal's Power Spectrum")

    for key, val in dict_integrated_test_signal.items():
        display.display_waveshow(val, key + "'s Test Signal's Waveform")
        display.display_power_spectrum(val, key + "'s Test Signal's Power Spectrum")

In [ ]:
# Feature Extraction (MFCC)
N_MFCC = 40
n_fft = int(SAMPLE_RATE * FRAME_LENGTH) # length of the FFT window, frame 하나당 sample의 수 / n_fft=2048, n_fft = int(SAMPLE_RATE * FRAME_LENGTH)
hop_length = int(SAMPLE_RATE * FRAME_STRIDE) # number of samples between successive frames / hop_length=512, hop_length = int(SAMPLE_RATE * FRAME_STRIDE)
n_mels = 128
f_max = 8000

dict_melspectrogram_train_validation_signal = {}
for key, val in dict_integrated_train_validation_signal.items():
    dict_melspectrogram_train_validation_signal[key] = librosa.feature.melspectrogram(val, sr=SAMPLE_RATE, n_mels=n_mels, fmax=f_max, n_fft=n_fft, hop_length=hop_length)

dict_melspectrogram_test_signal = {}
for key, val in dict_integrated_test_signal.items():
    dict_melspectrogram_test_signal[key] = librosa.feature.melspectrogram(val, sr=SAMPLE_RATE, n_mels=n_mels, fmax=f_max, n_fft=n_fft, hop_length=hop_length)

dict_mfccs_train_validation_signal = {}
for key, val in dict_melspectrogram_train_validation_signal.items():
    dict_mfccs_train_validation_signal[key] = librosa.feature.mfcc(S=librosa.power_to_db(val), sr=SAMPLE_RATE, n_mfcc=N_MFCC)

dict_mfccs_test_signal = {}
for key, val in dict_melspectrogram_test_signal.items():
    dict_mfccs_test_signal[key] = librosa.feature.mfcc(S=librosa.power_to_db(val), sr=SAMPLE_RATE, n_mfcc=N_MFCC)

if b_visualizing == True:
    for key, val in dict_mfccs_train_validation_signal.items():
        display.display_mfccs(val,  "'s Training+Validation Signal's MFCCs")
    
    for key, val in dict_mfccs_test_signal.items():
        display.display_mfccs(val,  "'s TEST Signal's MFCCs")

In [ ]:
arr_train_validation_data = None
arr_train_validation_target = None

for key, val in dict_mfccs_train_validation_signal.items():    
    arr_mfccs_data = val.transpose()    
    arr_target = np.full((arr_mfccs_data.shape[0], 1), key)   

    if arr_train_validation_data is None:
        arr_train_validation_data = arr_mfccs_data
        arr_train_validation_target = arr_target        
    else:       
        arr_train_validation_data = np.vstack((arr_train_validation_data, arr_mfccs_data))
        arr_train_validation_target = np.vstack((arr_train_validation_target, arr_target))

arr_test_data = None
arr_test_target = None

for key, val in dict_mfccs_test_signal.items():    
    arr_mfccs_data = val.transpose()    
    arr_target = np.full((arr_mfccs_data.shape[0], 1), key)   

    if arr_test_data is None:
        arr_test_data = arr_mfccs_data
        arr_test_target = arr_target        
    else:       
        arr_test_data = np.vstack((arr_test_data, arr_mfccs_data))
        arr_test_target = np.vstack((arr_test_target, arr_target))

print("Training+Validation data / target : ", arr_train_validation_data.shape, arr_train_validation_target.shape)
print("Test data / target : ", arr_test_data.shape, arr_test_target.shape)

In [ ]:
arr_std_train_validation_data, arr_std_test_tata = classifier.set_scaler_train_test(arr_train_validation_data, arr_test_data)

In [ ]:
dict_scores = {'kernel': [], 'acc': [], 'tpr': [], 'fnr': [], 'tnr': [], 'fpr': []}
targets_name = preprocessor.get_target_names(TEST_DATASET_DIR)

# linear
svm_model = classifier.set_fit_svm_model("linear", arr_std_train_validation_data, arr_train_validation_target)
acc, tpr, fnr, tnr, fpr = classifier.predict_svm_classification(svm_model, arr_std_test_tata, arr_test_target, targets_name, now_date_time)
save_each_classification_performance(svm_model.kernel, targets_name, acc, tpr, fnr, tnr, fpr)

dict_scores['kernel'].append(svm_model.kernel)        
dict_scores['acc'].append(acc)
dict_scores['tpr'].append(round(np.mean(tpr), DIGITS))
dict_scores['fnr'].append(round(np.mean(fnr), DIGITS))
dict_scores['tnr'].append(round(np.mean(tnr), DIGITS))
dict_scores['fpr'].append(round(np.mean(fpr), DIGITS))

# LinearSVC
svm_model = classifier.set_fit_svm_model("LinearSVC", arr_std_train_validation_data, arr_train_validation_target)
acc, tpr, fnr, tnr, fpr = classifier.predict_svm_classification(svm_model, arr_std_test_tata, arr_test_target, targets_name, now_date_time)

save_each_classification_performance("LinearSVC", targets_name, acc, tpr, fnr, tnr, fpr)

dict_scores['kernel'].append("LinearSVC")        
dict_scores['acc'].append(acc)
dict_scores['tpr'].append(round(np.mean(tpr), DIGITS))
dict_scores['fnr'].append(round(np.mean(fnr), DIGITS))
dict_scores['tnr'].append(round(np.mean(tnr), DIGITS))
dict_scores['fpr'].append(round(np.mean(fpr), DIGITS))

# rbf
svm_model = classifier.set_fit_svm_model("rbf", arr_std_train_validation_data, arr_train_validation_target)
acc, tpr, fnr, tnr, fpr = classifier.predict_svm_classification(svm_model, arr_std_test_tata, arr_test_target, targets_name, now_date_time)

save_each_classification_performance(svm_model.kernel, targets_name, acc, tpr, fnr, tnr, fpr)

dict_scores['kernel'].append(svm_model.kernel)        
dict_scores['acc'].append(acc)
dict_scores['tpr'].append(round(np.mean(tpr), DIGITS))
dict_scores['fnr'].append(round(np.mean(fnr), DIGITS))
dict_scores['tnr'].append(round(np.mean(tnr), DIGITS))
dict_scores['fpr'].append(round(np.mean(fpr), DIGITS))

# poly
svm_model = classifier.set_fit_svm_model("poly", arr_std_train_validation_data, arr_train_validation_target)
acc, tpr, fnr, tnr, fpr = classifier.predict_svm_classification(svm_model, arr_std_test_tata, arr_test_target, targets_name, now_date_time)

save_each_classification_performance(svm_model.kernel, targets_name, acc, tpr, fnr, tnr, fpr)

dict_scores['kernel'].append(svm_model.kernel)        
dict_scores['acc'].append(acc)
dict_scores['tpr'].append(round(np.mean(tpr), DIGITS))
dict_scores['fnr'].append(round(np.mean(fnr), DIGITS))
dict_scores['tnr'].append(round(np.mean(tnr), DIGITS))
dict_scores['fpr'].append(round(np.mean(fpr), DIGITS))

# sigmoid
svm_model = classifier.set_fit_svm_model("sigmoid", arr_std_train_validation_data, arr_train_validation_target)
acc, tpr, fnr, tnr, fpr = classifier.predict_svm_classification(svm_model, arr_std_test_tata, arr_test_target, targets_name, now_date_time)

save_each_classification_performance(svm_model.kernel, targets_name, acc, tpr, fnr, tnr, fpr)

dict_scores['kernel'].append(svm_model.kernel)        
dict_scores['acc'].append(acc)
dict_scores['tpr'].append(round(np.mean(tpr), DIGITS))
dict_scores['fnr'].append(round(np.mean(fnr), DIGITS))
dict_scores['tnr'].append(round(np.mean(tnr), DIGITS))
dict_scores['fpr'].append(round(np.mean(fpr), DIGITS))

save_total_classification_performance(dict_scores)

In [ ]:
# classifier.cross_validation_svm("linear", x_train_std, y_train)
# classifier.gridsearchCV_svm("linear", x_train_std, y_train)